In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from layers.Embed import DataEmbedding
from layers.Conv_Blocks import Inception_Block_V1
from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_csv('../../Dataset/Gold.csv')
data

,Open,Close,High,Low,Date
0,1302.910,1316.500,1318.910,1302.855,1/2/2018
1,1318.375,1318.875,1321.300,1307.710,1/3/2018
2,1313.105,1327.875,1327.875,1306.050,1/4/2018
3,1322.050,1319.590,1323.505,1313.910,1/5/2018
4,1321.050,1320.980,1322.210,1315.105,1/8/2018
...,...,...,...,...,...
2110,2352.200,2361.310,2363.460,2340.420,5/28/2024
2111,2361.390,2339.110,2361.790,2335.070,5/29/2024
2112,2338.880,2343.380,2351.090,2323.030,5/30/2024
2113,2343.340,2327.820,2359.580,2320.630,5/31/2024


In [5]:
# Chuyển cột 'Date' thành kiểu datetime
data['Date'] = pd.to_datetime(data['Date'])

In [6]:
prices = data['Close'].values.reshape(-1, 1)

In [7]:
# Chuẩn hóa dữ liệu
scaler = StandardScaler()
prices = scaler.fit_transform(prices)

In [8]:
# Chia dữ liệu thành tập train và tập test (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(prices, prices, test_size=0.3, shuffle=False)

In [9]:
# Chuyển dữ liệu thành Tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

In [10]:
# Định nghĩa lớp Dataset cho dữ liệu
class TimeSeriesDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [11]:
# Tạo các đối tượng DataLoader cho tập train và tập test
train_dataset = TimeSeriesDataset(X_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)

test_dataset = TimeSeriesDataset(X_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [12]:
# Xây dựng Timesblock
class TimesBlock(nn.Module):
    # Định nghĩa TimesBlock
    def __init__(self, configs):
        super(TimesBlock, self).__init__()
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
        self.k = configs.top_k
        # parameter-efficient design
        self.conv = nn.Sequential(
            Inception_Block_V1(configs.d_model, configs.d_ff,
                               num_kernels=configs.num_kernels),
            nn.GELU(),
            Inception_Block_V1(configs.d_ff, configs.d_model,
                               num_kernels=configs.num_kernels)
        )

    def forward(self, x):
        B, T, N = x.size()
        period_list, period_weight = FFT_for_Period(x, self.k)

        res = []
        for i in range(self.k):
            period = period_list[i]
            # padding
            if (self.seq_len + self.pred_len) % period != 0:
                length = (((self.seq_len + self.pred_len) // period) + 1) * period
                padding = torch.zeros(B, (length - (self.seq_len + self.pred_len)), N).to(x.device)
                out = torch.cat([x, padding], dim=1)
            else:
                length = (self.seq_len + self.pred_len)
                out = x
            # reshape
            out = out.reshape(B, length // period, period, N).permute(0, 3, 1, 2).contiguous()
            # 2D conv: from 1d Variation to 2d Variation
            out = self.conv(out)
            # reshape back
            out = out.permute(0, 2, 3, 1).reshape(B, -1, N)
            res.append(out[:, :(self.seq_len + self.pred_len), :])
        res = torch.stack(res, dim=-1)
        # adaptive aggregation
        period_weight = F.softmax(period_weight, dim=1)
        period_weight = period_weight.unsqueeze(1).unsqueeze(1).repeat(1, T, N, 1)
        res = torch.sum(res * period_weight, -1)
        # residual connection
        res = res + x
        return res

In [13]:
#Xây dựng mô hình Timesnet
class Model(nn.Module):
    # Định nghĩa Model
    def __init__(self, configs):
        super(Model, self).__init__()
        self.configs = configs
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
        self.layer = configs.e_layers  # Sửa tên biến từ 'configs.e_layers' thành 'self.layer'
        self.model = nn.ModuleList([TimesBlock(configs) for _ in range(self.layer)])  # Sửa 'configs' thành 'self.layer'
        self.enc_embedding = DataEmbedding(configs.enc_in, configs.d_model, configs.embed, configs.freq,
                                           configs.dropout)
        self.layer_norm = nn.LayerNorm(configs.d_model)
        self.predict_linear = nn.Linear(self.seq_len, self.pred_len + self.seq_len)
        self.projection = nn.Linear(configs.d_model, configs.c_out, bias=True)

    def forecast(self, x_enc, x_mark_enc):
        means = x_enc.mean(1, keepdim=True).detach()
        x_enc = x_enc - means
        stdev = torch.sqrt(torch.var(x_enc, dim=1, keepdim=True, unbiased=False) + 1e-5)
        x_enc /= stdev

        enc_out = self.enc_embedding(x_enc, x_mark_enc)  # [B,T,C]
        enc_out = self.predict_linear(enc_out.permute(0, 2, 1)).permute(0, 2, 1)  # align temporal dimension

        for i in range(self.layer):
            enc_out = self.layer_norm(self.model[i](enc_out))

        dec_out = self.projection(enc_out)

        dec_out = dec_out * (stdev[:, 0, :].unsqueeze(1).repeat(1, self.pred_len + self.seq_len, 1))
        dec_out = dec_out + (means[:, 0, :].unsqueeze(1).repeat(1, self.pred_len + self.seq_len, 1))
        
        return dec_out


In [14]:
class MyConfig:
    def __init__(self, seq_len, pred_len, top_k, layer, dmodel, dff, num, enc, embed, freq, dropout, c_out):
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.top_k = top_k
        self.e_layers = layer
        self.d_model = dmodel
        self.d_ff = dff
        self.num_kernels = num
        self.enc_in = enc
        self.embed = embed
        self.freq = freq
        self.dropout = dropout
        self.c_out = c_out
configs = MyConfig(seq_len=150, pred_len=90, top_k=3, layer=2, dmodel=16, dff=32, num=32, enc=7, embed='fixed', freq='B', dropout=0.2, c_out=7)

In [15]:
# Khởi tạo mô hình
model = Model(configs)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [16]:
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for batch_x in train_loader:
        batch_x = batch_x.to(device)
        optimizer.zero_grad()
        output = model.forecast(batch_x)
        loss = loss_fn(output, batch_x)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    print(f'Epoch {epoch+1}, Loss: {epoch_loss}')

TypeError: forecast() missing 1 required positional argument: 'x_mark_enc'